In [ ]:
import pandas as pd




In [ ]:
df = pd.read_excel('/content/translated_column.xlsx')

In [ ]:
df.head()

,text
0,Ravish kumar sir ji sir ji aap pen pen pekin k...
1,Hi broker
2,22:20 Ravish ji already knows this thing.
3,Chal jhuthae
4,https://youtu.be/6bozyHzbIys?si=4JvRt5DB8ba6Hz-a


In [ ]:
!pip install emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 10.4 MB/s eta 0:00:00


In [ ]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=4d5f1135c455c0f9d52d603a21c47079bba59131f92d79bbb0a90368201e028b
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import pandas as pd
import re
import emoji
import requests
from langdetect import detect, LangDetectException


# Function to clean text
def clean_text(text):
    if isinstance(text, str):  # Ensure the input is a string
        text = text.lower()
        text = re.sub(r'http\S+', '', text)  # Remove links
        text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove mentions
        text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Remove hashtags
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = emoji.demojize(text)  # Convert emojis to text
        text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    else:
        text = ''
    return text

# Function to detect if text is in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False

# Preprocess the comments and filter out non-English comments
df['cleaned_comments'] = df['text'].apply(clean_text)
df = df[df['cleaned_comments'].apply(is_english)]



In [ ]:
df.head()

,text,cleaned_comments
2,22:20 Ravish ji already knows this thing.,2220 ravish ji already knows this thing
3,Chal jhuthae,chal jhuthae
8,"Ravish ji, say something about trains Sir ji M...",ravish ji say something about trains sir ji ma...
9,Jal gai app ki hai. What will you say after th...,jal gai app ki hai what will you say after the...
10,BJP only BJP,bjp only bjp


In [ ]:



#!pip install hatesonar


In [ ]:
'''from hatesonar import Sonar

sonar = Sonar()

def get_hate_speech_label(text):
    result = sonar.ping(text)
    if result['top_class'] == 'hate_speech':
        return 'hate'
    else:
        return 'not hate'

df['label'] = df['cleaned_comments'].apply(get_hate_speech_label)'''


"from hatesonar import Sonar\n\nsonar = Sonar()\n\ndef get_hate_speech_label(text):\n    result = sonar.ping(text)\n    if result['top_class'] == 'hate_speech':\n        return 'hate'\n    else:\n        return 'not hate'\n\ndf['label'] = df['cleaned_comments'].apply(get_hate_speech_label)"

In [ ]:
!pip install detoxify transformers


In [ ]:
'''from detoxify import Detoxify

def get_hate_speech_label(text):
    results = Detoxify('original').predict(text)
    if results['toxicity'] > 0.5:  # You can adjust this threshold based on your needs
        return 'hate'
    else:
        return 'not hate'

df['label'] = df['cleaned_comments'].apply(get_hate_speech_label)
'''

"from detoxify import Detoxify\n\ndef get_hate_speech_label(text):\n    results = Detoxify('original').predict(text)\n    if results['toxicity'] > 0.5:  # You can adjust this threshold based on your needs\n        return 'hate'\n    else:\n        return 'not hate'\n\ndf['label'] = df['cleaned_comments'].apply(get_hate_speech_label)\n"

In [ ]:
import pandas as pd
import re
import emoji
from concurrent.futures import ThreadPoolExecutor, as_completed
from detoxify import Detoxify



# Function to clean text
def clean_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'http\S+', '', text)  # Remove links
        text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove mentions
        text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Remove hashtags
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = emoji.demojize(text)  # Convert emojis to text
        text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    else:
        text = ''
    return text

# Clean the comments
df['cleaned_comments'] = df['text'].apply(clean_text)

# Function to get hate speech label
def get_hate_speech_label(text):
    results = Detoxify('original').predict(text)
    if results['toxicity'] > 0.5:  # Adjust threshold as needed
        return 'hate'
    else:
        return 'not hate'

# Function to process a chunk of the dataframe
def process_chunk(chunk):
    chunk['label'] = chunk['cleaned_comments'].apply(get_hate_speech_label)
    return chunk

# Split the dataframe into chunks
chunk_size = 100  # Adjust based on your memory and system capability
chunks = [df.iloc[i:i + chunk_size].copy() for i in range(0, df.shape[0], chunk_size)]

# Process the chunks in parallel
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_chunk, chunk) for chunk in chunks]
    results = []
    first_chunk_processed = False
    for future in as_completed(futures):
        result = future.result()
        results.append(result)
        if not first_chunk_processed:
            print("First batch processed:")
            print(result.head())
            first_chunk_processed = True

# Concatenate the results back into a single dataframe
df_labeled = pd.concat(results)

# Save the labeled data
df_labeled.to_excel('labeled_comments.xlsx', index=False)

# Also save individual chunks
for i, chunk in enumerate(chunks):
    chunk.to_excel(f'labeled_comments_chunk_{i + 1}.xlsx', index=False)



Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt
Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt
Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt
Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt
Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt
Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt"

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


First batch processed:
                                                  text  \
327             You may not get a heart attack, crows.   
331  Today these newspaper exit polls are in favor ...   
332  🤣🤣🤣🤣🤣 Abbey, you go first and sit here in the ...   
334  In Namaskar, Modi's blind opponent Ravish Kuma...   
337  Now why will the exit poll give 400 seats to C...   

                                      cleaned_comments     label  
327               you may not get a heart attack crows      hate  
331  today these newspaper exit polls are in favor ...  not hate  
332   abbey you go first and sit here in the room t...  not hate  
334  in namaskar modis blind opponent ravish kumar ...  not hate  
337  now why will the exit poll give 400 seats to c...  not hate  


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in tra

In [ ]:
df_labeled.head()

,text,cleaned_comments,label
327,"You may not get a heart attack, crows.",you may not get a heart attack crows,hate
331,Today these newspaper exit polls are in favor ...,today these newspaper exit polls are in favor ...,not hate
332,"🤣🤣🤣🤣🤣 Abbey, you go first and sit here in the ...",abbey you go first and sit here in the room t...,not hate
334,"In Namaskar, Modi's blind opponent Ravish Kuma...",in namaskar modis blind opponent ravish kumar ...,not hate
337,Now why will the exit poll give 400 seats to C...,now why will the exit poll give 400 seats to c...,not hate


In [ ]:
df_labeled.nunique()

,0
text,18027
cleaned_comments,17818
label,2


In [ ]:
# unique values
df_labeled['label'].value_counts()

,count
label,
not hate,17437
hate,1043


In [ ]:
hc=df[df_labeled['label']=='hate']

<ipython-input-23-7a34bfca1336>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hc=df[df_labeled['label']=='hate']


In [ ]:
print(hc)

                                                    text  \
20                      Ravish bhai Kejriwal is a thief.   
46                      Ravish bhai Kejriwal is a thief.   
54     Gong's donkeys cleverly obtained votes by givi...   
93           You guard, you are a threat to the country.   
102                         Only BJP will come you idiot   
...                                                  ...   
25920                              This heinous Indian,,   
25944       This guy is a big fraud, stay away from him.   
25969                             Drink the public crazy   
25976  Rathi ji, he is the biggest traitor, he will k...   
26019  It was fun BC Modi became PM but get the opera...   

                                        cleaned_comments  
20                       ravish bhai kejriwal is a thief  
46                       ravish bhai kejriwal is a thief  
54     gongs donkeys cleverly obtained votes by givin...  
93             you guard you are a threat t